In [1]:
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import time
torch.backends.cudnn.benchmark = True
import os

In [2]:
size_pad = 30

In [4]:
# Modification of the original loader

import torch
from torch.utils.data import Dataset
import polars as pl


class CalorichEventDataset(Dataset):
    def __init__(self, event_with_hit_features_path, hits_path):
        """Constructor"""
        self.event_with_hit_features_columns = [
            'composite_event_id',  # Remember to drop
            'ring_radius_cal'      # Added the target, to separate later
            # 'total_in_time_hits'
        ]

        self.event_with_hit_features = (
            pl
            .read_parquet(event_with_hit_features_path)
            .select(self.event_with_hit_features_columns)
            .drop_nulls()
            .head(1000)
            .filter(pl.col('ring_radius_cal').is_not_nan())
        )

        self.hits_columns = [
            'composite_event_id',  # Remember to drop
            'x_adjusted', 'y_adjusted'
        ]

        self.hits = (
            pl
            .scan_parquet(hits_path)
            .select(self.hits_columns)
        )

    def __len__(self):
        """Size of the dataset"""
        return self.event_with_hit_features.shape[0]

    def __getitem__(self, idx):
        """Get a particular item of the dataset"""

        row = self.event_with_hit_features.row(idx)
        composite_event_id = row[0]
        hits = (
        self.hits
        .filter(pl.col("composite_event_id") == composite_event_id)
        .drop("composite_event_id")
        .collect()
        )

        # Add 3rd dimension as gaussian noise

        noise = np.random.normal(0, 0.05, 30)
        noise = np.expand_dims(noise, 1)
        

        return dict(zip(
        self.event_with_hit_features_columns[1:],
        torch.tensor(
            [n for n in self.event_with_hit_features.row(idx)[1:]])
        )) | {
            # Hits is now a n*3 tensor array
            "hits": torch.tensor(np.hstack([hits.sample(n=size_pad, seed=42, with_replacement=True).rows(), noise])).to(device)
        }

# Load Dataset

In [5]:
dataset = CalorichEventDataset(r'C:\Users\merig\OneDrive\Documentos\Projects\PointNet\data\events_with_hit_features_[cut_off_time=0.5].parquet', 
                               r'C:\Users\merig\OneDrive\Documentos\Projects\PointNet\data\hits.parquet')

In [6]:
dataset.__len__()

1000

In [7]:
dataset.__getitem__(0)

{'ring_radius_cal': tensor(173.4102),
 'hits': tensor([[-3.4970e+02, -2.2774e+02, -2.2288e-02],
         [-3.3170e+02, -2.5892e+02, -3.9044e-02],
         [-3.5870e+02, -1.4980e+02,  2.4509e-03],
         [-3.7670e+02, -2.4333e+02,  4.0280e-02],
         [-1.4270e+02, -2.5090e+01, -3.9893e-02],
         [-3.4970e+02, -2.2774e+02,  2.3448e-02],
         [-3.3170e+02, -2.5892e+02, -3.1577e-02],
         [-5.2700e+01, -2.4333e+02,  2.7645e-02],
         [-7.9700e+01, -2.9009e+02, -7.0915e-02],
         [-3.1370e+02, -4.0680e+01,  1.6554e-02],
         [-3.1370e+02, -4.0680e+01,  3.8165e-02],
         [-1.6700e+01, -1.8097e+02, -1.8169e-02],
         [-1.2470e+02,  6.0900e+00,  4.7826e-02],
         [-1.6700e+01, -1.1862e+02, -1.8540e-02],
         [-9.7700e+01, -2.9009e+02, -6.3188e-02],
         [-3.5870e+02, -1.4980e+02,  2.5080e-02],
         [-9.7700e+01, -2.9009e+02,  2.2734e-02],
         [-1.6700e+01, -1.8097e+02,  6.1609e-03],
         [-1.2470e+02,  6.0900e+00,  4.9652e-02],
    

In [8]:
len(dataset.__getitem__(0)['hits'])

30

## Save Tensors to Disk

In [9]:
test_name = 'this_test'
torch.save(dataset.__getitem__(0), os.path.join(locale, f'{test_name}.pt'))


KeyboardInterrupt



In [34]:
locale = r'C:\Users\merig\Documents\Projects\PointNet\data\tensors'

In [35]:
head = dataset.__len__()

In [36]:
for i in range(head):
    torch.save(dataset.__getitem__(i), os.path.join(locale, f'{i}.pt'))

In [44]:
torch.load(os.path.join(locale, f'{0}.pt'))['ring_radius_cal']

tensor(173.4102)

In [40]:
test_dict = torch.load(os.path.join(locale, f'{0}.pt'))

In [42]:
test_dict.update(torch.load(os.path.join(locale, f'{1}.pt')))

In [43]:
test_dict

{'ring_radius_cal': tensor(174.6605),
 'hits': tensor([[-3.4970e+02, -2.2774e+02, -8.5467e-02],
         [-3.9470e+02, -2.1215e+02, -8.6957e-02],
         [-3.9470e+02, -2.1215e+02, -3.7858e-03],
         [-3.7670e+02, -2.4333e+02,  1.1493e-02],
         [-3.1370e+02,  8.4030e+01, -2.4771e-02],
         [-3.7670e+02, -2.1215e+02,  4.8322e-02],
         [-2.7770e+02,  1.1521e+02,  1.7689e-03],
         [-1.2880e+02, -1.6010e+02, -2.2489e-03],
         [-2.0570e+02, -1.9656e+02, -4.6417e-02],
         [-4.8470e+02, -5.6270e+01, -7.3546e-03],
         [-4.8470e+02, -5.6270e+01, -1.3740e-01],
         [-1.4270e+02, -1.1862e+02, -4.8820e-03],
         [-3.4970e+02,  1.1521e+02,  1.0032e-02],
         [-2.2370e+02, -1.9656e+02, -4.9081e-02],
         [-2.3270e+02, -2.4333e+02, -3.8641e-02],
         [-3.1370e+02,  8.4030e+01, -3.0103e-02],
         [-1.6070e+02, -1.1862e+02,  1.2037e-01],
         [-1.4270e+02, -1.1862e+02, -3.5665e-02],
         [-3.4970e+02,  1.1521e+02,  1.4165e-02],
    

In [49]:
a = torch.load(os.path.join(locale, f'{0}.pt'))['hits']
b = torch.load(os.path.join(locale, f'{1}.pt'))['hits']
test_cat = torch.cat((a, b), 0)

In [50]:
test_cat

tensor([[-3.4970e+02, -2.2774e+02, -2.4646e-02],
        [-3.3170e+02, -2.5892e+02, -1.8230e-02],
        [-3.5870e+02, -1.4980e+02, -8.9511e-03],
        [-3.7670e+02, -2.4333e+02,  3.1408e-02],
        [-1.4270e+02, -2.5090e+01, -2.2773e-02],
        [-3.4970e+02, -2.2774e+02, -5.5075e-02],
        [-3.3170e+02, -2.5892e+02, -3.1802e-02],
        [-5.2700e+01, -2.4333e+02,  6.7771e-02],
        [-7.9700e+01, -2.9009e+02, -4.9130e-02],
        [-3.1370e+02, -4.0680e+01,  8.1258e-02],
        [-3.1370e+02, -4.0680e+01,  4.9169e-03],
        [-1.6700e+01, -1.8097e+02, -4.0932e-02],
        [-1.2470e+02,  6.0900e+00,  1.9684e-02],
        [-1.6700e+01, -1.1862e+02, -6.5588e-03],
        [-9.7700e+01, -2.9009e+02, -4.4177e-02],
        [-3.5870e+02, -1.4980e+02, -5.3689e-02],
        [-9.7700e+01, -2.9009e+02,  5.4376e-02],
        [-1.6700e+01, -1.8097e+02,  4.9624e-02],
        [-1.2470e+02,  6.0900e+00, -5.5119e-02],
        [-3.4970e+02, -2.2774e+02,  1.0583e-01],
        [-3.5870e+02

# Disk Loader

In [10]:
class EmbedDataset(torch.utils.data.Dataset):
    def __init__(self, path):
        self.path = path 

        

    def __len__(self):
        return len(self.path)

    def __getitem__(self, i):

        tensor = torch.load(os.path.join(self.path, f'{i}.pt'))

        return tensor

In [23]:
dataset = EmbedDataset(locale)

In [25]:
dataset.__len__()

AttributeError: 'EmbedDataset' object has no attribute 'labels'

# PadCollate

In [8]:
def pad_tensor(vec, pad, dim):
    """
    args:
        vec - tensor to pad
        pad - the size to pad to
        dim - dimension to pad

    return:
        a new tensor padded to 'pad' in dimension 'dim'
    """
    pad_size = list(vec.shape)
    pad_size[dim] = pad - vec.size(dim)
    return torch.cat([vec, torch.zeros(*pad_size)], dim=dim)


class PadCollate:
    """
    a variant of callate_fn that pads according to the longest sequence in
    a batch of sequences
    """

    def __init__(self, dim=0):
        """
        args:
            dim - the dimension to be padded (dimension of time in sequences)
        """
        self.dim = dim

    def pad_collate(self, batch):
        """
        args:
            batch - list of (tensor, label)

        reutrn:
            xs - a tensor of all examples in 'batch' after padding
            ys - a LongTensor of all labels in batch
        """
        # find longest sequence
        max_len = max(map(lambda x: x[0].shape[self.dim], batch))
        # pad according to max_len
        batch = map(lambda x, y:
                    (pad_tensor(x, pad=max_len, dim=self.dim), y), batch)
        # stack all
        xs = torch.stack(map(lambda x: x[0], batch), dim=0)
        ys = torch.LongTensor(map(lambda x: x[1], batch))
        return xs, ys

    def __call__(self, batch):
        return self.pad_collate(batch)


# DataLoader

In [12]:
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np

batch_size = 10
validation_split = 0.2
shuffle_dataset = True
random_seed = 42
num_workers = 0
pin_memory = False


indices = list(range(dataset.__len__()))
split = int(np.floor(validation_split * dataset.__len__()))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler, num_workers = num_workers, pin_memory = pin_memory)
validation_loader = DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler, num_workers = num_workers, pin_memory = pin_memory)

In [32]:
train_loader

# PointNet

In [11]:
"""
PointNet implementation for the RICH AI project.

Adapted from:
https://github.com/charlesq34/pointnet (Author implementation)
http://stanford.edu/~rqi/pointnet/ (Original paper)
"""

import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class STNkd(nn.Module):
    """TNet Implementation in PyTorch.
    k x k transformation matrix predicted by T-Net to coordinates of input points.
    """

    def __init__(self, k=64):
        super().__init__()
        self.k = k
        self.conv1 = nn.Conv1d(k, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k * k)
        self.relu = nn.ReLU()

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)

        self.iden = Variable(
            torch.from_numpy(np.eye(self.k).flatten().astype(np.float32))
        ).view(1, -1)

    def forward(self, x):
        batchsize = x.size()[0]
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = nn.MaxPool1d(x.size(-1))(x)
        x = nn.Flatten(1)(x)
        x = self.relu(self.bn4(self.fc1(x)))
        x = self.relu(self.bn5(self.fc2(x)))

        # initialize as identity
        iden = torch.eye(self.k, requires_grad=True).repeat(batchsize, 1, 1)
        iden = iden.to(device)
        x = self.fc3(x).view(-1, self.k, self.k) + iden

        return x


class Transform(nn.Module):
    """Input and Feature Transform module."""

    def __init__(self):
        super().__init__()
        self.input_transform = STNkd(k=3)
        self.feature_transform = STNkd(k=64)
        self.conv1 = nn.Conv1d(3, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)
        self.relu = nn.ReLU()
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)

    def forward(self, x):

        # batch matrix multiplication
        # input transform
        x = torch.bmm(torch.transpose(x, 1, 2), self.input_transform(x)).transpose(1, 2)

        x = self.relu(self.bn1(self.conv1(x)))

        # Feature transform
        x = torch.bmm(torch.transpose(x, 1, 2), self.feature_transform(x)).transpose(
            1, 2
        )

        x = self.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = nn.MaxPool1d(x.size(-1))(x)
        x = x.view(-1, 1024)
        return x


class PointNetFc(nn.Module):
    """PointNet fully connected network.

    Attributes
    ----------
    num_classes : int
        Number of classes in the classification network.
    momentum : bool
        If True, include momentum as feature.
    radius : bool
        If True, include radius as feature.

    Methods
    -------
    forward(x, momentum, radius)
        Feed forward nn layer with input x, momentum and radius.
    """

    def __init__(
        self,
        num_classes,
        momentum=False,
        radius=False,
    ):
        super().__init__()
        self.num_classes = num_classes
        self.momentum = momentum
        self.radius = radius

        self.feat = Transform()

        # include radius and momentum
        if self.momentum and self.radius:
            self.fc1 = nn.Linear(1024 + 2, 512)
        elif self.momentum or self.radius:
            self.fc1 = nn.Linear(1024 + 1, 512)
        else:
            self.fc1 = nn.Linear(1024, 512)

        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, self.num_classes)

        self.dropout = nn.Dropout(p=0.3)

        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(p=0.3)
        self.relu = nn.ReLU()

    def forward(self, x, momentum=None, radius=None):

        x = x.transpose(1, 2)

        x = self.feat(x)

        # add momentum dimension if desired
        if self.momentum and momentum is not None:
            x = torch.hstack([x, momentum.unsqueeze(1)])

        # add radius dimension if desired
        if self.radius and radius is not None:
            x = torch.hstack([x, radius.unsqueeze(1)])

        x = self.relu(self.bn1(self.fc1(x)))
        x = self.relu(self.bn2(self.dropout(self.fc2(x))))
        x = self.fc3(x)

        return x

### Jack's Pointnet

In [12]:
import torch
from torch import nn


class TNet(nn.Module):
    def __init__(self, k=64):
        super(TNet, self).__init__()
        self.k = k

        self.conv_network = nn.Sequential(
            nn.Conv1d(k, 64, 1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Conv1d(64, 128, 1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Conv1d(128, 1024, 1),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
        )
        self.fc_network = nn.Sequential(
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, k * k)
        )

    def forward(self, x):
        batch_size = x.size(0)
        x = self.conv_network(x)
        x = nn.AdaptiveMaxPool1d(1)(x)
        x = x.view(batch_size, -1)
        x = self.fc_network(x)

        identity_matrix = torch.eye(self.k, device=x.device).view(1, self.k * self.k).repeat(batch_size, 1)
        matrix = x + identity_matrix
        return matrix.view(batch_size, self.k, self.k)


class PointNet(nn.Module):
    def __init__(self, input_dim=3, output_dim=1024):
        super(PointNet, self).__init__()
        self.input_transform = TNet(k=input_dim)
        self.feature_transform = TNet(k=64)

        self.input_network = nn.Sequential(
            nn.Conv1d(input_dim, 64, 1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Conv1d(64, 64, 1),
            nn.BatchNorm1d(64),
            nn.ReLU()
        )

        self.feature_network = nn.Sequential(
            nn.Conv1d(64, 64, 1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Conv1d(64, 128, 1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Conv1d(128, output_dim, 1),
            nn.BatchNorm1d(output_dim),
            nn.ReLU()
        )

        self.conv1 = nn.Conv1d(input_dim, 64, 1)
        self.conv2 = nn.Conv1d(64, 64, 1)
        self.conv3 = nn.Conv1d(64, 64, 1)
        self.conv4 = nn.Conv1d(64, 128, 1)
        self.conv5 = nn.Conv1d(128, output_dim, 1)

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(64)
        self.bn3 = nn.BatchNorm1d(64)
        self.bn4 = nn.BatchNorm1d(128)
        self.bn5 = nn.BatchNorm1d(output_dim)

    def forward(self, x):
        batch_size = x.size(0)

        # Transpose input data to match expected dimensions
        x = x.transpose(2, 1)

        input_transform = self.input_transform(x)
        x = torch.bmm(x.transpose(1, 2), input_transform).transpose(1, 2)

        x = self.input_network(x)

        feature_transform = self.feature_transform(x)
        x = torch.bmm(x.transpose(1, 2), feature_transform).transpose(1, 2)

        x = self.feature_network(x)
        x = nn.AdaptiveMaxPool1d(1)(x)
        x = x.view(batch_size, -1)

        return x

## PointNet for Regression

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PointNetRegression(nn.Module):
    def __init__(self, num_features, num_output):
        super(PointNetRegression, self).__init__()
        
        # Input transformation network
        self.input_transform = nn.Sequential(
            nn.Conv1d(num_features, 64, kernel_size=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Conv1d(64, 128, kernel_size=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Conv1d(128, 1024, kernel_size=1),
            nn.BatchNorm1d(1024),
            nn.ReLU()
        )
        
        # Feature transformation network
        self.feature_transform = nn.Sequential(
            nn.Conv1d(1024, 512, kernel_size=1),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Conv1d(512, 256, kernel_size=1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Conv1d(256, 128, kernel_size=1),
            nn.BatchNorm1d(128),
            nn.ReLU()
        )
        
        # Fully connected layers for regression
        self.fc_regression = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, num_output)
        )

    def forward(self, x):
        batch_size = x.size(0)
        
        # Input transformation network
        x = self.input_transform(x)
        
        # Global feature extraction
        x = torch.max(x, 2, keepdim=True)[0]
        
        # Feature transformation network
        x = self.feature_transform(x)
        
        # Global feature vector
        x = torch.max(x, 2, keepdim=False)[0]
        
        # Fully connected layers for regression
        x = self.fc_regression(x)
        
        return x


# Trainer

In [14]:
import torch

print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(f"ID of current CUDA device:{torch.cuda.current_device()}")
	
print(f"Name of current CUDA device:{torch.cuda.get_device_name(cuda_id)}")


Is CUDA supported by this system? True
CUDA version: 11.8
ID of current CUDA device:0
Name of current CUDA device:NVIDIA GeForce RTX 3080


## Second Trainer

In [20]:
def best_trainer(model, criterion, optimizer, trainloader, validloader, epochs = 5, patience = 5, verbose = True):

    train_loss = []
    valid_loss = []
    start_time = 0
    end_time = 0
    print(device)
    start_time = time.time()
    model.to(device)
    end_time = time.time()
    print(f'Model loading took {end_time - start_time:.3f} seconds')

    for epoch in range(epochs):

        train_batch_loss = 0
        valid_batch_loss = 0

        # Start the trainer

        for i, d in enumerate(trainloader):
            print(f'Iteration {i + 1}')
            start_time = time.time()
            X = d['hits'].float() # float32
            y = d['ring_radius_cal'].float().to(device) # float32
            y = y.unsqueeze(1)
            end_time = time.time()
            # print(f'Data loading time: {end_time - start_time:.3f}')

            # Model is used
            start_time = time.time()
            optimizer.zero_grad()
            predictions = model(X)
            end_time = time.time()
            # print(f'The model took {end_time - start_time:.3f} seconds to run with cuda = {X.is_cuda} and {X.type()} training dataset and cuda = {y.is_cuda} and {y.type()} ground truth dataset')

            # Loss
        
            start_time = time.time()
            loss = criterion(predictions, y)
            loss.backward()
            optimizer.step()
            end_time = time.time()
            # print(f'The loss process took {end_time - start_time:.3f} seconds')
            train_batch_loss += loss.item()

        train_loss.append(loss.item() / len(train_loader))

        # Validation Loop

        with torch.no_grad(): # Stops graph computations

            for X_valid, y_valid in enumerate(validloader):
                start_time = time.time()
                X_valid = d['hits'].float() # float32
                y_valid = d['ring_radius_cal'].float().to(device) # float32
                y_valid = y_valid.unsqueeze(1)
                valid_preds = model(X_valid)
                loss = criterion(valid_preds, y_valid)
                end_time = time.time()
                # print(f'Validation loop took {end_time - start_time:.3f} seconds')

                valid_batch_loss += loss.item()

            valid_loss.append(valid_batch_loss / len(validation_loader))

        if verbose:
            print(f'Epoch {epoch + 1}',
                  f'Train loss: {train_loss[-1]:.3f}'
                  f'Validation loss: {valid_loss[-1]:.3f}')
    
    return train_loss, valid_loss

### Parameters for Training

In [21]:
model = PointNetRegression(30, 1)
learning_rate = 0.001
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
epochs = 5

# Train

In [22]:
%%time
train_loss, valid_loss = best_trainer(model, 
                                 criterion, 
                                 optimizer, 
                                 train_loader,
                                 validation_loader,
                                 epochs=5, 
                                 patience=5,
                                 verbose=True)

cuda
Model loading took 0.024 seconds
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5


KeyboardInterrupt: 

In [23]:
print(device)
model.to(device)
trainTemplate = "epoch: {} test loss: {:.3f} test accuracy: {:.3f}"
training_start = time.time()
for epoch in range(epochs):
    print("[INFO] epoch: {}...".format(epoch + 1))
    epoch_start = time.time()
    train_losses, train_accs = [], []
    samples = 0

    model.train()

    for i, d in enumerate(train_loader):
        X = d['hits'].float().to(device) # Check why the change into a different datatype and why send it to device
        y = d['ring_radius_cal'].to(device) # Remember, I removed the .long() before .to(device) to get the decimal points (does not make sense)
        predictions = model(X)
        loss = criterion(predictions, y) # erased long() from y.long()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item() * y.size(0)) # Changed to append
        train_accs.append(torch.sub(predictions[0], y).mean().item()) # Added the () in sum()
        samples += y.size(0)

        print(f"Epoch: {epoch + 1} step: {i} with loss: {loss.item()} and accuracy: {train_accs[i]}")


cuda
[INFO] epoch: 1...


KeyboardInterrupt: 

In [ ]:
for i, d in enumerate(validation_loader, 0):
    y = d['ring_radius_cal']
    X = d['hits']

    print(f"size of d: {len(d.items())}" )
    print(f"value of i: {i}" )
    print(f"size of X: {len(X)}")
    print(f"size of y: {len(y)}")


    # print(X.size())
    # print(y.size())
    # print(X)
    break

size of d: 2
value of i: 0
size of X: 16
size of y: 16


In [ ]:
for i, d in enumerate(train_loader, 0):
    y = d['ring_radius_cal']
    X = d['hits']

    print(f"size of d: {len(d.items())}" )
    print(f"value of i: {i}" )
    print(f"size of X: {len(X)}")
    print(f"size of y: {len(y)}")


    # print(X.size())
    # print(y.size())
    # print(X)
    break

size of d: 2
value of i: 0
size of X: 16
size of y: 16


In [ ]:
next(iter(dataset))

{'ring_radius_cal': tensor(173.4102),
 'hits': tensor([[-3.4970e+02, -2.2774e+02, -4.6963e-02],
         [-3.3170e+02, -2.5892e+02,  9.1718e-02],
         [-3.5870e+02, -1.4980e+02, -6.1191e-02],
         [-3.7670e+02, -2.4333e+02, -1.5209e-02],
         [-1.4270e+02, -2.5090e+01,  5.6494e-03],
         [-3.4970e+02, -2.2774e+02,  3.5191e-02],
         [-3.3170e+02, -2.5892e+02,  8.9147e-02],
         [-5.2700e+01, -2.4333e+02, -5.1090e-02],
         [-7.9700e+01, -2.9009e+02, -3.3477e-03],
         [-3.1370e+02, -4.0680e+01,  6.4508e-03],
         [-3.1370e+02, -4.0680e+01, -7.5159e-02],
         [-1.6700e+01, -1.8097e+02, -3.8311e-02],
         [-1.2470e+02,  6.0900e+00,  3.8668e-03],
         [-1.6700e+01, -1.1862e+02, -7.5247e-02],
         [-9.7700e+01, -2.9009e+02, -1.8152e-03],
         [-3.5870e+02, -1.4980e+02,  3.3626e-02],
         [-9.7700e+01, -2.9009e+02,  2.6931e-03],
         [-1.6700e+01, -1.8097e+02, -3.2215e-02],
         [-1.2470e+02,  6.0900e+00, -1.1841e-02],
    

In [ ]:
noise = np.random.normal(0, 0.05, 30)
noise = np.expand_dims(noise, 1)
noise.shape

(30, 1)

# Tests

In [51]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.transforms as transforms


model = models.resnet50()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

dataset = datasets.FakeData(
    size=1000,
    transform=transforms.ToTensor())
loader = DataLoader(
    dataset,
    num_workers=1,
    pin_memory=True
)

model.to('cuda')

for data, target in loader:
    data = data.to('cuda', non_blocking=True)
    target = target.to('cuda', non_blocking=True)
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    
print('Done')

Done


In [45]:
# Python code to merge dict using update() method
def Merge(dict1, dict2):
	return(dict2.update(dict1))


# Driver code
dict1 = {'a': 10, 'b': 8}
dict2 = {'d': 6, 'c': 4}

# This returns None
print(Merge(dict1, dict2))

# changes made in dict2
print(dict2)


None
{'d': 6, 'c': 4, 'a': 10, 'b': 8}
